In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\husey\appdata\local\temp\pip-req-build-unzhvg50
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101669 sha256=128dee8584ea6b77db7b9fbd3802bc863a3a799c49fbae972f011fd73e2dd51b
  Stored in directory: C:\Users\husey\AppData\Local\Temp\pip-ephem-wheel-cache-qgw4ju5w\wheels\67\d2\f4\96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
Successfully built keras-contrib


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers  import Activation, Conv2D, Input, Dropout, LeakyReLU, Input, Dense, Concatenate, Conv2DTranspose, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization 
from random import random, randint
from keras.models import load_model
import keras_contrib

In [3]:
def discriminatorModel(imageShape):
    init = RandomNormal(stddev=0.02)
    LInputImage = Input(shape=imageShape)
    disTensor = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(LInputImage)
    disTensor = LeakyReLU(alpha=0.2)(disTensor)
    disTensor = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(disTensor)
    disTensor = InstanceNormalization(axis=-1)(disTensor)
    disTensor = LeakyReLU(alpha=0.2)(disTensor)
    disTensor = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(disTensor)
    disTensor = InstanceNormalization(axis=-1)(disTensor)
    disTensor = LeakyReLU(alpha=0.2)(disTensor)
    disTensor = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(disTensor)
    disTensor = InstanceNormalization(axis=-1)(disTensor)
    disTensor = LeakyReLU(alpha=0.2)(disTensor)
    disTensor = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(disTensor)
    disTensor = InstanceNormalization(axis=-1)(disTensor)
    disTensor = LeakyReLU(alpha=0.2)(disTensor)
    LOut = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(disTensor)
    model = Model(LInputImage, LOut)
    model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
    return model

In [4]:
def residualBlock(LNumberofFilters, LInputLayer):
    init = RandomNormal(stddev=0.02)
    resTensor = Conv2D(LNumberofFilters, (3,3), padding='same', kernel_initializer=init)(LInputLayer)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Activation('relu')(resTensor)
    resTensor = Conv2D(LNumberofFilters, (3,3), padding='same', kernel_initializer=init)(resTensor)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Concatenate()([resTensor, LInputLayer])
    return resTensor

In [5]:
def generatorModel(LImageShape, LResnet=9):
    init = RandomNormal(stddev=0.02)
    LInputShape = Input(shape=LImageShape)
    genTensor = Conv2D(64, (7,7), padding = 'same', kernel_initializer=init)(LInputShape)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer=init)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(258, (3,3), strides = (2,2), padding = 'same', kernel_initializer=init)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    
    for _ in range(LResnet):
        genTensor = residualBlock(256, genTensor)
    
    genTensor = Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same', kernel_initializer=init)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same', kernel_initializer=init)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2D(3, (7,7), padding='same', activation='tanh', kernel_initializer=init)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    
    LOutImage = Activation('tanh')(genTensor)
    model = Model(LInputShape, LOutImage)
    return model

In [6]:
def compositeModel(generatorModel1, discriminatorModel, generatorModel2, imageShape):
    generatorModel1.trainable = True
    discriminatorModel.trainable = False
    generatorModel2.trainable = False

    InputGenerator = Input(shape=imageShape)
    generatorModel1Out = generatorModel1(InputGenerator)
    output_d = discriminatorModel(generatorModel1Out)

    input_id = Input(shape=imageShape)
    output_id = generatorModel1(input_id)

    output_f = generatorModel2(generatorModel1Out)
    generatorModel2Out = generatorModel2(input_id)
    output_b = generatorModel1(generatorModel2Out)

    model = Model([InputGenerator, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model